# Connecting with the Credo AI Governance Platform

## Overview
Connecting with the Credo AI Governance Platform is straightforward using Lens. After following this tutorial you'll be up and running!

### Find the code
This notebook can be found on [github](https://github.com/credo-ai/credoai_lens/blob/develop/docs/notebooks/governance_integration.ipynb).

**Table of Contents**

1. [Setup](#Setup)
1. [Using Lens](#Using-Lens)
    1. [Exporting to Credo AI's Governance Platform from Lens](#Exporting-to-Credo-AI's-Governance-Platform-from-Lens)
    2. [CredoGovernance](#Credo-Governance-Artifact)
    3. [Registering a model while assessing it](#Registering-a-model-while-assessing-it)
1. [Air Gap Environment](#Air-Gap-Environment)
    1. [Getting the assessment spec](#Getting-the-assessment-spec) 
    2. [Exporting Results](#Exporting-Results)
1. [Explicitly registering model and data artifacts](#Explicitly-registering-model-and-data-artifacts)
    1. [Registering a Project and Model](#Registering-a-Project-and-Model)

### Config File
First, ensure you have a config file somewhere on your computer named ".credoconfig". The default location where Lens will look for it in your home directory (`~/.credoconfig`). The structure of the config should look like this

```
TENANT=<tenant name> # Example: credoai
API_KEY=<your api key> # Example: JSMmd26...
```

This config gives Lens API access to the Governance Platform.

### Connecting to a particular model, dataset, or Use Case

Lens connects to the Governance Platform via a `CredoGovernance` object. This specifies the IDs of the Use Case, model and data you want to associate your assessments with. The IDs are found in the url of your Use Case or model.

For instance, the Use Case ID is found here: `...credo.com/use-cases/{use_case_id}`

And the model Id is found here: `...credo.com/models/{model_id}`

## Train your Model

Some quick setup... This is all of your datascience work before assessment and integration with Credo AI.

In [ ]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier

# Base Lens imports
import credoai.lens as cl

In [ ]:
data = fetch_creditdefault()
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']

model = GradientBoostingClassifier()
model.fit(X,y)

## Using Lens

This is going to be familiar if you've gone through our quickstart tutorial! The only addition is the use of a `CredoGovernance` object.

### Credo Governance Artifact
The CredoGovernance object has a few functionalities.

* The ability to get an alignment spec from the Use Case (created during the aligned process). The alignment spec is either downloaded straight from the Governance Platform, or supplied as a json file for air gap deployment. This alignment spec can be supplemented (or overwritten) with an alignment spec you define in code and pass to Lens.
* The ability to register models and projects.
* Encoding the IDs of governance objects so Lens can easily export metrics and reports.

\** **Attention** \**

The metrics and reports will _either_ be exported to the model and data specified in by `CredoGovernance` (using the `model_id` and `data_id` arguments) OR a new model and data will be registered on your CredoGovernance platform automatically, which will then be used for export.

With that said, let's see how this works in practice.

In [ ]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          X=X, 
                          y=y.astype(int),
                          sensitive_features=sensitive_feature)

# New artifact for governance
credo_governance = cl.CredoGovernance(use_case_id="your-use-case-id",
                                      model_id="your-model-id"
                                     )

In [ ]:
# specify the metrics that will be used by the FairnessBase assessment. 
# This spec will supplement (or overide!) whatever spec is 
# pulled down from the Governance Platform
alignment_spec = {
    'FairnessBase': {'metrics': ['precision_score']}
}

### Interacting with Credo AI's Governance Platform from Lens

In addition to the model and data, we can pass the CredoGovernance object. This is how Lens knows which Use Case, model and/or data to interacts with. If you are running assessments on multiple models, you'll create a separate `CredoGovernance` object (and separate Lens instances) for each.

That's it!

In [ ]:
from credoai.assessment import FairnessBaseAssessment
lens = cl.Lens(model=credo_model,
               governance=credo_governance,
               assessments = [FairnessBaseAssessment()],
               data=credo_data,
               spec=alignment_spec)

In [ ]:
# just add the export option and the metrics will be sent to the Governance Platform.
lens.run_assessments(export=True)

In [ ]:
# reports can be exported too!
lens.create_reports(export=True)

### Registering a model while assessing it

**No Governance Info? No Problem**

Sometimes you may not want to create a `CredoGovernance` object before using Lens. There may not be an `assessment spec` that you have to pull down for instance. Or you may not have a model registered yet in the Governance Platform to receive your assessments.

If you still want to log a model assessment, Lens will take care of registering a new model and project on Credo AI's Governance Platform first. The below will create a project called `an example model project` and log the model `an example model ` under it.

In [ ]:
from credoai.assessment import FairnessBaseAssessment
credo_model = cl.CredoModel(name='an example model',
                            model=model)

lens = cl.Lens(model=credo_model,
               assessments = [FairnessBaseAssessment()],
               data=credo_data,
               spec=alignment_spec)
lens.run_assessments(export=True)
lens.create_reports(export=True)

The governance object is created for you, and can be interrogated or used later.

In [ ]:
gov = lens.get_governance()
gov.get_info()

\** **Attention** \**


Note that if you run the above a second time, registration will fail. Project and model names must be unique! If you would like to run Lens again, sending the results to the same project/model you can get the governance info by running `lens.get_governance`, and passing that to the governance object of another Lens instance.

For example:

In [ ]:
"""
previously_created_governance = lens.get_governance()
lens = cl.Lens(model=credo_model,
               assessments = [FairnessBaseAssessment()],
               data=credo_data,
               spec=alignment_spec,
               governance = previously_created_governance)
"""

## Air Gap Environment

If you cannot interact with Credo AI's Governance Platfrom via the internet, this section is for you!

Instead of Lens automating the communication between your assessment environment and governance, you will instead have to download assets and upload them. 

### Getting the assessment spec
The asset that you may have to get _from_ the governance platform is the assessment spec. This can be found under the "Technical Requirements" of your Use Case. Once you download the assessments spec, you can read it with a CredoGovernance object:




In [ ]:
gov = cl.CredoGovernance()
# you must explicitly retrieve the assessment spec 
# in an air gapped environment!
gov.retrieve_assessment_spec('{path to assessment spec}')

Following that, you can pass the governance object to Lens as normal.

### Exporting Results

Lens can export assessment results to file as a json object. This is generally useful if you want to store your results locally, but particularly useful if you are in an air gapped environment where you cannot access Credo AI's Governance Platform directly from your development environment.

Doing this only requires a one line change in our current flow. We change the `export` argument from `True` to a local path:


In [ ]:
lens.run_assessments(export='{path where assessments should be exported}')

# reports are exported to file in an analagous way.
# lens.create_reports(export='{path where assessments should be exported}')

Running the above will create a folder that you specified, if it doesn't exist. Within that folder each assessment's output will be saved as a separate json file. This json file can be uploaded by going to the **Asset Store** of your Model on the Governance Platform, pressing the purple "+" button under "Metrics" and uploading the json. Reports should be uploaded under "Chart".

## Explicitly registering model and data artifacts

While Lens will take care of registering a model and data for you if no `model_id` or `data_id` is provided, you can also manually register models yourself using a `CredoGovernance` object.

This is helpful if you want to run Lens multiple times, or want to have more control over the names of your project and/or model. 
* First, you create a CredoGovernance object
* Second register your project and model
* Third, pass to Lens as normal

### Registering a Project and Model

**Models** are organized under **Projects**. If you register a Model, a Project will be automatically created to house the Model.

In [ ]:
gov = cl.CredoGovernance()
gov.register_model(model_name='my_model')

If a Project already exists that you'd like to use (registering a model _to_ that project), supply the Project ID to the `CredoGovernance` object

In [ ]:
gov = cl.CredoGovernance(model_project_id = 'existing project')
gov.register_model(model_name='my_model')

If you want to register a new project and control its name, register the Project explicitly before registering the Model.

Note - if the model already exists, and is associated with a project, the project will be registered without a model! (The model already is registered to another project).

In [ ]:
gov = cl.CredoGovernance()
gov.register_project('my_project')
gov.register_model(model_name='my_model')

**Datasets** can be registered analogously.

In [ ]:
gov.register_dataset('my_data')

**Use Cases** can also be used. If CredoGovernance has a Use Case ID, the model will automatically be associated with that Use Case ID

In [ ]:
gov = cl.CredoGovernance(use_case_id_id="your-use-case-id")
gov.register_model(model_name='my_model')